
<br>
<font>
<div dir=ltr align=center>
<img src="https://cdn.freebiesupply.com/logos/large/2x/sharif-logo-png-transparent.png" width=150 height=150> <br>
<font color=0F5298 size=7>
Artificial Intelligence <br>
<font color=2565AE size=5>
Computer Engineering Department <br>
Spring 2024<br>
<font color=3C99D size=5>
Practical Assignment - Minimax <br>
<font color=696880 size=4>
Ali Aghayari


____

In [1]:
name = 'Mohammad Emad Changizi Ashtiani'
student_number = '401105826'

# P0 : Game explanation and environment setup (0 points)

In this Jupyter notebook, we aim to develop the AI logic for the game **Connect4**. Players take turns dropping pieces into a grid, and the first player to align four consecutive pieces vertically, horizontally, or diagonally wins. The focus is on creating an intelligent AI opponent using the minimax algorithm with alpha-beta pruning to deliver a challenging gameplay experience.

<span style="color: red;">However, there’s an added twist: after either player drops a piece, there is a 12.5% chance that the entire board will rotate 90 degrees clockwise, changing the direction of gravity as well! Keep this rule in mind when implementing your heuristic, as your AI will face a tough challenge ahead.</span>

<br>

Note: The winning condition will be checked after applying the rotation. If both sides have winning conditions, the player who made the last move will lose.

<br>
Note: For full clarification, the following occurs:
- Some players make their moves.
- Rotations happens. (12.5%)
- New gravity is applied to every piece on the board. (12.5%)
- The winning condition is then checked.

<br>
Rules: Do not modify the code cells that don’t have TODO comments, except for those that are explicitly mentioned as okay to change.

First, we will define some constants to make the code cleaner, more organized, and to set up the game environment.

In [2]:
# Game Constants
ROW_COUNT = 7
COLUMN_COUNT = 7
WINDOW_LENGTH = 4
EMPTY = 0

# Players and Pieces
PLAYER = 0
AI = 1
EMPTY_PIECE = 0
PLAYER_PIECE = 1
AI_PIECE = 2

# Colors (RGB values) - you can change the colors to your liking
FG_COLOR = (0, 0, 255)
BG_COLOR = (0, 0, 0)
P1_COLOR = (255, 0, 0)
P2_COLOR = (0, 255, 0)

# Pygame Constants
SQUARESIZE = 80
RADIUS = SQUARESIZE // 2 - 5
SCREEN_WIDTH = COLUMN_COUNT * SQUARESIZE
SCREEN_HEIGHT = (ROW_COUNT + 1) * SQUARESIZE

These libraries are sufficient to complete this task, but feel free to add any additional imports you may need.

In [3]:
%pip install pygame

import numpy as np
import random
import pygame
import math
import time
import copy

# Additional libraries can be imported here if needed


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Emad\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


pygame 2.6.1 (SDL 2.28.4, Python 3.10.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


# P1 : Util functions (20 points)

We need to initialize the game board as an empty 2D array with dimensions of ROW_COUNT by COLUMN_COUNT.

In [4]:
def create_board():
  board = [[EMPTY_PIECE for _ in range(COLUMN_COUNT)] for _ in range(ROW_COUNT)]
  return board
    # TODO: Follow the instructions as described.
    # return board

Fill in the code to find the valid columns where a piece can be dropped.

In [5]:
def is_valid_location(board, col):
  valid = get_valid_locations(board)
  return valid[col] == True
    # TODO: Follow the instructions as described.
    # return True/False

In [6]:
def get_valid_locations(board):
  valid_loc = [any(row[i] == EMPTY_PIECE for row in board) for i in range(COLUMN_COUNT)]
  return valid_loc
    # TODO: Follow the instructions as described.
    # return valid locations

Fill in the code to find the valid row for the given column where a piece can be dropped.

In [7]:
def get_next_open_row(board, col):
  for row in range(ROW_COUNT):
    if board[row][col] == EMPTY_PIECE:
      return row
    # TODO: Follow the instructions as described.
    # return row

Fill in the code to drop a piece in the specified column of the board.

In [8]:
def drop_piece(board, col, piece):
  x = get_next_open_row(board, col)
  board[x][col] = piece
    # TODO: Follow the instructions as described.
    # no return needed

Fill in the code to check if the specified piece has won the game.

In [9]:
def winning_move(board, piece):

  def check_four(r1, c1, r2, c2, r3, c3, r4, c4):
    if board[r1][c1] == piece and board[r1][c1] == board[r2][c2] == board[r3][c3] == board[r4][c4]:
      return True
    return False

  for r in range(ROW_COUNT):
    for c in range(COLUMN_COUNT):
      #check horizontal
      if c <= COLUMN_COUNT - 4:
        result = check_four(r, c, r, c + 1, r, c + 2, r, c + 3)
        if result:
          return True

      #check vertical
      if r <= ROW_COUNT - 4:
        result = check_four(r, c, r + 1, c, r + 2, c, r + 3, c)
        if result:
          return True

      #check diagonal
      if r <= ROW_COUNT - 4 and c <= COLUMN_COUNT - 4:
        result = check_four(r, c, r + 1, c + 1, r + 2, c + 2, r + 3, c + 3)
        if result:
          return True


      #check anti-diagonal
      if r <= ROW_COUNT - 4 and c >= 3:
        result = check_four(r, c, r + 1 , c - 1, r + 2, c - 2, r + 3, c - 3)
        if result:
          return True

  return False

    # TODO: Follow the instructions as described.
    # return True/False

# P2 : Scoring function and Minimax implementation (50 points)

Fill in the code to score the current situation for the player. Hint: You can divide the board into separate windows, score each window individually for the given piece, and sum the scores to obtain the total board score for that piece.

In [10]:
def evaluate_window(window, piece):
  score = 0
  if window.count(piece) == 4:
    score += 1000
  elif window.count(piece) == 3:
    score += 100
  elif window.count(piece) == 2:
    score += 20
  elif window.count(piece) == 1:
    score += 10

  score *= -1 if piece == PLAYER_PIECE else 1
  return score

    # TODO: Follow the instructions as described.
    # NOTE: You can implement the scoring as you see fit
    # return score

In [11]:
def score_position(board, piece):
  score = 0

  #score horizontal window
  for r in range(ROW_COUNT):
    for c in range(COLUMN_COUNT - 3):
      window = [board[r][c + i] for i in range(4)]
      score += evaluate_window(window, piece)

  #score vertical window
  for c in range(COLUMN_COUNT):
    for r in range(ROW_COUNT - 3):
      window = [board[r + i][c] for i in range(4)]
      score += evaluate_window(window, piece)

  #score diagonal window
  for r in range(ROW_COUNT - 3):
    for c in range(COLUMN_COUNT - 3):
      window = [board[r + i][c + i] for i in range(4)]
      score += evaluate_window(window, piece)

  #score anti-diagonal window
  for r in range(ROW_COUNT - 3):
    for c in range(3, COLUMN_COUNT):
      window = [board[r + i][c - i] for i in range(4)]
      score += evaluate_window(window, piece)

  return score
    # TODO: Follow the instructions as described.
    # NOTE: Split the board into separate windows and sum up the scores for each window.
    # NOTE: Adjust the scoring logic to account for the additional game rules.
    # return aggregated score

Fill in the code to implement the minimax algorithm with alpha beta pruning using the utility function provided.

In [12]:
def minimax(board, depth, alpha, beta, maximizingPlayer):
  piece = AI_PIECE if maximizingPlayer else PLAYER_PIECE
  val_loc = get_valid_locations(board)
  best_col = None

  #check the terminal conditions
  if winning_move(board, piece) and piece == AI_PIECE:
    return best_col, 1000000
  elif winning_move(board, piece) and piece == PLAYER_PIECE:
    return best_col, -1000000
  elif depth == 0:
    return best_col, score_position(board, piece)
  elif not any(val_loc): #there is no valid column left so game is over and no one wins
    return best_col, 0   # return 0 for draw situation

  if maximizingPlayer:
    max_val = -math.inf
    for col in range(COLUMN_COUNT):
      if is_valid_location(board, col):
        next_row = get_next_open_row(board, col)
        temp_board = copy.deepcopy(board)
        temp_board[next_row][col] = piece

        new_score = minimax(temp_board, depth - 1, alpha, beta, False)[1]
        if new_score > max_val:
          max_val = new_score
          best_col = col
        alpha = max(alpha, max_val)
        if alpha >= beta:
          break
    return best_col, max_val

  else:
    min_val = math.inf
    for col in range(COLUMN_COUNT):
      if is_valid_location(board, col):
        next_row = get_next_open_row(board, col)
        temp_board = copy.deepcopy(board)
        temp_board[next_row][col] = piece

        new_score = minimax(temp_board, depth - 1, alpha, beta, True)[1]
        if new_score < min_val:
          min_val = new_score
          best_col = col
        beta = min(beta, min_val)
        if alpha >= beta:
          break

    return best_col, min_val

    # TODO: Retrieve the list of valid moves
    # TODO: Return the appropriate value if a terminal condition (win/loss/draw/max_depth) is met
    # TODO: Check whose turn it is, and apply minimax logic for that player; recursively call with decreased depth for the opponent
    # TODO: Implement alpha-beta pruning to optimize the search
    # NOTE: Be careful not to alter the original board during recursive exploration
    # NOTE: This function is supposed to find the best column to drop a piece on
    # Return best column to drop and the value associated to this move
    # Return best_column, value
    pass

# P3 : PVE (0 points)

Run this code to test your AI's performance. It is recommended to execute this part locally.

In [13]:
pygame.init()
width = COLUMN_COUNT * SQUARESIZE
height = (ROW_COUNT + 1) * SQUARESIZE
size = (width, height)
screen = pygame.display.set_mode(size)
myfont = pygame.font.SysFont("monospace", 75)

def rotate_board(board):
    rotated_board = np.rot90(board)
    for col in range(rotated_board.shape[1]):
        column = rotated_board[:, col]
        non_empty = column[column != 0]
        empty_count = ROW_COUNT - len(non_empty)
        rotated_board[:, col] = np.concatenate((non_empty, np.zeros(empty_count)))
    return rotated_board

def draw_board(board):
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT):
            pygame.draw.rect(screen, FG_COLOR, (c * SQUARESIZE, r * SQUARESIZE + SQUARESIZE, SQUARESIZE, SQUARESIZE))
            pygame.draw.circle(screen, BG_COLOR, (
                int(c * SQUARESIZE + SQUARESIZE / 2), int(r * SQUARESIZE + SQUARESIZE + SQUARESIZE / 2)), RADIUS)

    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT):
            if board[r][c] == PLAYER_PIECE:
                pygame.draw.circle(screen, P1_COLOR, (
                    int(c * SQUARESIZE + SQUARESIZE / 2), height - int(r * SQUARESIZE + SQUARESIZE / 2)), RADIUS)
            elif board[r][c] == AI_PIECE:
                pygame.draw.circle(screen, P2_COLOR, (
                    int(c * SQUARESIZE + SQUARESIZE / 2), height - int(r * SQUARESIZE + SQUARESIZE / 2)), RADIUS)
    pygame.display.update()

def run_game():
    board = create_board()
    draw_board(board)
    game_over = False
    turn = random.choice([0, 1])
    while not game_over:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return

            if event.type == pygame.MOUSEMOTION:
                pygame.draw.rect(screen, BG_COLOR, (0, 0, width, SQUARESIZE))
                posx = event.pos[0]
                if turn == PLAYER:
                    pygame.draw.circle(screen, P1_COLOR, (posx, int(SQUARESIZE / 2)), RADIUS)

            pygame.display.update()

            if event.type == pygame.MOUSEBUTTONDOWN:
                pygame.draw.rect(screen, BG_COLOR, (0, 0, width, SQUARESIZE))
                if turn == PLAYER:
                    posx = event.pos[0]
                    col = int(math.floor(posx / SQUARESIZE))

                    if is_valid_location(board, col):
                        drop_piece(board, col, PLAYER_PIECE)
                        if random.random() <= 0.125:
                            board = rotate_board(board)
                            draw_board(board)
                            pygame.time.wait(1500)

                        if winning_move(board, AI_PIECE):
                            label = myfont.render("Player 2 wins!!", 1, P2_COLOR)
                            screen.blit(label, (40, 10))
                            game_over = True
                        elif winning_move(board, PLAYER_PIECE):
                            label = myfont.render("Player 1 wins!!", 1, P1_COLOR)
                            screen.blit(label, (40, 10))
                            game_over = True

                        turn += 1
                        turn = turn % 2
                        draw_board(board)

        if turn == AI and not game_over:
            col, minimax_score = minimax(board, 5, -math.inf, math.inf, True)
            if is_valid_location(board, col):
                drop_piece(board, col, AI_PIECE)

                if random.random() <= 0.125:
                    board = rotate_board(board)
                    draw_board(board)
                    pygame.time.wait(1500)

                if winning_move(board, PLAYER_PIECE):
                    label = myfont.render("Player 1 wins!!", 1, P1_COLOR)
                    screen.blit(label, (40, 10))
                    game_over = True
                elif winning_move(board, AI_PIECE):
                    label = myfont.render("Player 2 wins!!", 1, P2_COLOR)
                    screen.blit(label, (40, 10))
                    game_over = True
                turn += 1
                turn = turn % 2
                draw_board(board)

        if game_over:
            pygame.time.wait(3000)
            pygame.quit()
            return


run_game()

# P4 : EVE (30 points)

In this section, we will simulate an AI battle where your AI heuristic should outperform our provided heuristic. Don’t worry; the opposing AI is not optimal, but if your scoring approach is inadequate, you may lose some credit from P2. Your AI should demonstrate a significant advantage, meaning it should consistently beat our AI on average, regardless of whether it plays as the first or second player. Please note that your search tree should can't have a higher depth than our heuristic.

Implement the minimax algorithm similar to your main minimax function. However, do not modify the tester_evaluate_window and tester_score_position functions. Remember to utilize tester_score_position within tester_minimax!

In [14]:
def tester_evaluate_window(window, piece):
    score = 0
    opp_piece = PLAYER_PIECE if piece == AI_PIECE else AI_PIECE
    if window.count(piece) == 4:
        score += 100
    elif window.count(piece) == 3 and window.count(EMPTY) == 1:
        score += 5
    elif window.count(piece) == 2 and window.count(EMPTY) == 2:
        score += 2
    if window.count(opp_piece) == 3 and window.count(EMPTY) == 1:
        score -= 4
    return score


def tester_score_position(board, piece):
    score = 0
    center_array = [int(i) for i in list(board[:, COLUMN_COUNT // 2])]
    score += center_array.count(piece) * 3

    for r in range(ROW_COUNT):
        row_array = [int(i) for i in list(board[r, :])]
        for c in range(COLUMN_COUNT - 3):
            window = row_array[c:c + WINDOW_LENGTH]
            score += tester_evaluate_window(window, piece)

    for c in range(COLUMN_COUNT):
        col_array = [int(i) for i in list(board[:, c])]
        for r in range(ROW_COUNT - 3):
            window = col_array[r:r + WINDOW_LENGTH]
            score += tester_evaluate_window(window, piece)

    for r in range(ROW_COUNT - 3):
        for c in range(COLUMN_COUNT - 3):
            window = [board[r + i][c + i] for i in range(WINDOW_LENGTH)]
            score += tester_evaluate_window(window, piece)

    for r in range(ROW_COUNT - 3):
        for c in range(COLUMN_COUNT - 3):
            window = [board[r + 3 - i][c + i] for i in range(WINDOW_LENGTH)]
            score += tester_evaluate_window(window, piece)


def tester_minimax(board, depth, alpha, beta, maximizingPlayer):
  piece = AI_PIECE if maximizingPlayer else PLAYER_PIECE
  val_loc = get_valid_locations(board)
  best_col = None

  #check the terminal conditions
  if winning_move(board, piece) and piece == AI_PIECE:
    return best_col, 1000000
  elif winning_move(board, piece) and piece == PLAYER_PIECE:
    return best_col, -1000000
  elif depth == 0 and not maximizingPlayer:
    return best_col, tester_score_position(board, piece)
  elif depth == 0 and maximizingPlayer:
    return best_col, score_position(board, piece)
  elif not any(val_loc): #there is no valid column left so game is over and no one wins
    return best_col, 0   # return 0 for draw situation

  if maximizingPlayer:
    max_val = -math.inf
    for col in range(COLUMN_COUNT):
      if is_valid_location(board, col):
        next_row = get_next_open_row(board, col)
        temp_board = copy.deepcopy(board)
        temp_board[next_row][col] = piece

        new_score = minimax(temp_board, depth - 1, alpha, beta, False)[1]
        if new_score > max_val:
          max_val = new_score
          best_col = col
        alpha = max(alpha, max_val)
        if alpha >= beta:
          break
    return best_col, max_val

  else:
    min_val = math.inf
    for col in range(COLUMN_COUNT):
      if is_valid_location(board, col):
        next_row = get_next_open_row(board, col)
        temp_board = copy.deepcopy(board)
        temp_board[next_row][col] = piece

        new_score = minimax(temp_board, depth - 1, alpha, beta, True)[1]
        if new_score < min_val:
          min_val = new_score
          best_col = col
        beta = min(beta, min_val)
        if alpha >= beta:
          break

    return best_col, min_val


    # TODO: Implement using your main minimax logic
    # NOTE: Remember to replace score_position with tester_score_position!!!
    # NOTE: In this function, you represent the "AI" and the tester represents the "PLAYER".
    #       No further changes are needed if you haven't modified the constants.
    pass

In [15]:
def simulate_game():
    board = create_board()
    starting_turn = turn = random.choice([0, 1])

    game_over = False
    while not game_over:
        if turn == PLAYER:
            col, minimax_score = tester_minimax(board, 4, -math.inf, math.inf, True)
            if is_valid_location(board, col):
                drop_piece(board, col, PLAYER_PIECE)
                if random.random() <= 0.125:
                    board = rotate_board(board)

                if winning_move(board, AI_PIECE):
                    return 1, starting_turn

                elif winning_move(board, PLAYER_PIECE):
                    return 0, starting_turn
                turn += 1
                turn = turn % 2

        if turn == AI and not game_over:
            col, minimax_score = minimax(board, 4, -math.inf, math.inf, True)
            if is_valid_location(board, col):
                drop_piece(board, col, AI_PIECE)
                if random.random() <= 0.125:
                    board = rotate_board(board)
                if winning_move(board, PLAYER_PIECE):
                    return 0, starting_turn
                elif winning_move(board, AI_PIECE):
                    return 1, starting_turn

                turn += 1
                turn = turn % 2

Run this tester. You need to win at least 65% of the games to pass.
<br>
The code execution should take less than 10 minutes to complete. If it exceeds this time, performance optimization might be necessary.

In [16]:
wins = 0
tests = 100
throw = 0
disadvantage = 0

start_time = time.time()

for i in range(tests):
    result = simulate_game()
    if result[1] == PLAYER and result[0] == 0: disadvantage += 1
    if result[1] == AI and result[0] == 0: throw += 1
    wins += result[0]

end_time = time.time()

print(f"you won {wins / tests}% of games")
print(f"you throw {throw / tests}% of games")
print(f"you lost logically {disadvantage / tests}% of games")
print(f"Code execution time: {end_time - start_time:.4f} seconds")

you won 0.74% of games
you throw 0.15% of games
you lost logically 0.11% of games
Code execution time: 500.6966 seconds
